## Correlated and Biased Simulated Measurement Errors on BV ##
This notebook runs a number of experiments using different measurement error mitigation strategies against an implementation of the Bernstein-Vazirani algorithm where all oracles produce states in the measurement basis.

In [ ]:
# qiskit does not play nicely with modern numpy
import warnings

warnings.catch_warnings() 
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

# Mainline Python Imports
import random
import math
import numpy as np
import copy
import sys, os, time

from typing import Callable
from functools import reduce


# Plotting Imports
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sbs

sbs.set(style="darkgrid")
%matplotlib inline

from qutip import *
import scipy

# Qiskit Imports
import qiskit
from qiskit import IBMQ
from qiskit import transpile, QuantumRegister, assemble
from qiskit import QuantumCircuit, execute, Aer, QuantumCircuit
from qiskit.ignis.mitigation.measurement import complete_meas_cal, tensored_meas_cal, CompleteMeasFitter, TensoredMeasFitter
import qiskit.ignis.verification.randomized_benchmarking as rb

IBMQ.load_account()

### A quick and dirty progress bar ###

In [ ]:
from math import floor

class Pbar():
    '''
        Simple progress bar class
    '''
    
    def __init__(
            self, 
            n_ticks, # Number of ticks to print
            n_calls, # Number of times you expect to call this progress bar
            name='', # Name of bar to display
            ticker='=', # Ticker symbol(s)
            ticker_head='>', # Head of the ticker
            ticker_blank=' '): # Unticked symbol(s)
        self.n_ticks = n_ticks
        self.n_calls = n_calls
        self.name = name
        self.ticker=ticker
        self.ticker_head = ticker_head
        self.ticker_blank = ticker_blank
        self.p_len = 0 # Length of previous print
        self.invoked = 0 # Has it been called yet?

    def __call__(self, message=''):


        print('\b' * self.p_len  , end='', flush=True) # Flush previous print

        ticker = floor(self.invoked / self.n_calls * self.n_ticks) * self.ticker
        blank = (self.n_ticks - len(ticker) - 1) * self.ticker_blank
        head = [self.ticker_head, ''][len(ticker) >= self.n_ticks]

        fstring = "\r{name} : [{ticker}{head}{blank}] {msg}".format(
                name = self.name,
                ticker = ticker,
                head = head,
                blank = blank, 
                msg = message
                )
        self.p_len = len(fstring) + 15
        print(fstring, end='', flush=True)
        self.invoked += 1 


A circuit constructor that applies a given inversion array and then adds measurements.
The inversion array is needed for AIM and SIM.

In [ ]:
def design_circuit(n_qubits, inv_arr, circuit=None):
    '''
        Given a circuit and an inversion array apply X on all 1 values in the array then measure all qubits
    '''
    
    # No circuit, make a bare one
    if circuit is None:
        circuit = QuantumCircuit(n_qubits, n_qubits)
    
    # Apply flips from inv_array
    for i, element in enumerate(inv_arr):
        if int(element) == 1:
            circuit.x(i)
    
    # Measure all qubits
    circuit.measure(list(range(n_qubits)), list(range(n_qubits)))
    
    return circuit

Constructs a BV circuit with an oracle that constructs the state described by the bv string

In [ ]:
def bv_circuit(bv_string : str, n_qubits : int):
    '''
        Returns a BV circuit that prepares the state bv_string over n_qubits
        :: bv_string ::
        :: n_qubits
    '''
    bv_circuit = QuantumCircuit(n_qubits, n_qubits - 1)

    # Hadamard on all participating qubits
    for i in range(n_qubits):
        bv_circuit.h(i)

    bv_circuit.z(n_qubits - 1)

    bv_circuit.barrier()

    # Oracle to construct state
    # Performs a CNOT from the target qubit to the ensemble
    for i in range(n_qubits - 1):
        if bv_string[i] == '1':
            bv_circuit.cx(i, n_qubits - 1)


    # Barrier to prevent optimisations over pairs of hadarmards, oracle is *supposed* to be blind
    bv_circuit.barrier()

    # Hadamard on all participating qubits
    for i in range(n_qubits - 1):
        bv_circuit.h(i)

    return bv_circuit

## Sampling Methods ##
For sampling simulated measurement errors

In [ ]:
def sample_distribution(population : dict, n_shots : int) -> dict:
    '''
        Distribution sampling method
        Given an existing set of results, resample
        Used for simulating measurement errors
        
        :: population : dict :: Current shot population to sample
        :: n_shots    : int  :: Number of shots to take on the population
    '''
    # There are much more efficient ways to do this
    
    n_counts = sum(population.values())
    vals = list(population.keys())
    weights = np.array([population[i] / n_counts for i in vals])
    
    weights /= sum(weights)
    
    updated_population = weighted_sample_distribution(weights, vals, n_shots)
    
    return updated_population

def weighted_sample_distribution(weights : list, values : list, n_shots : int) -> dict:
    '''
        weighted_sample_distribution
        Distribution sampling method
        Given an existing set of results and weights, resample
        Used for simulating measurement errors
        
        :: weights : list :: Current shot population to sample
        :: values  : list :: Values associated with each weight
        :: n_shots : int  :: Number of shots to take on the population
    '''
    out_distribution = {}
    
    # Normalise weights, just to be sure
    weights /= sum(weights)
    
    # Rough check on sum of weights
    assert(sum(weights) <= 1 or math.isclose(sum(weights), 1, abs_tol=1e-5))
    
    # Cumulative weights
    c_weights = [0]
    for weight in weights:
        c_weights.append(weight + c_weights[-1])

    # Take random shots at the distribution
    for _ in range(n_shots):
        v = np.random.random()

        # This would be faster with a BST or some fancy hashing
        for val, lower, upper in zip(values, c_weights[:-1], c_weights[1:]):
        
            # Check bound on random value
            if v > lower and v < upper: 
                if val in out_distribution:
                    out_distribution[val] += 1
                else:
                    out_distribution[val] = 1
                break
                
    return out_distribution

## Simulated Measurement Error Channel ##
Constructs measurement errors with statistical biases towards error weights and state dependence

In [ ]:
def gen_error_probs(
    error_arr_c : list, # Error distance weights
    error_arr_u : list, # Error state dependent weights up
    error_arr_d : list, # Error state dependent weights down
    n_qubits = 4):
    '''
        gen_error_probs
        
        Function to generate error probabilities for simulated measurement errors
        
        :: error_arr_c : list :: Weights on error sizes
        :: error_arr_u : list :: Weights on "up" state depentent errors
        :: error_arr_d : list :: Weights on "down" state depentent errors
        :: n_qubits    : int  :: Number of qubits
        
        Returns a measurement error channel
    
    '''
    
    # Build initial prob vector
    probs = [[0] * (2 ** n_qubits) for _ in range(2 ** n_qubits)]
    
    # Check size of weights
    if len(error_arr_c) != n_qubits + 1:
        raise Exception("Incorrect Error Array")
    
    if len(error_arr_u) != n_qubits + 1:
        raise Exception("Incorrect Error Array")
        
        
    if len(error_arr_d) != n_qubits + 1:
        raise Exception("Incorrect Error Array")
    
    # Fill vector
    for row in range(2 ** n_qubits):
        row_str = bin(row)[2:].zfill(n_qubits)


        for col in range(2 ** n_qubits):
            col_str = bin(col)[2:].zfill(n_qubits)

            diff_str = [i - j for i, j in zip(list(map(int, row_str)), list(map(int, col_str)))]   
            
            probs[row][col] += error_arr_u[sum(1 if i == -1 else 0 for i in diff_str)]
            probs[row][col] += error_arr_d[sum(1 if i == 1 else 0 for i in diff_str)]
            probs[row][col] += error_arr_c[n_qubits - sum(1 if i == 0 else 0 for i in diff_str)]
            
            probs[row][col] = max(0, probs[row][col])
            
    #Normalise rows, we can then do arbitrary scaling factors in the error arr
    for row, _ in enumerate(probs):
        np_row = np.array(probs[row])
        if sum(np_row) > 0:
            np_row = np_row / sum(np_row) 
        probs[row] = list(np_row)

    return probs

Example measurement error channel

In [ ]:
m = gen_error_probs(
        [100,30,0,0,0], # Const - Controls correlation of error weights
        [0, 5, 30, 5, 5], # 0 -> 1 - Controls state dependent errors
        [0, 0, 0, 0, 0] # 1 -> 0 - Controls state dependent errors
        )
sbs.heatmap(m)

### Measurement Error Application Functions ###
Act to apply measurement errors to dict representations or results representations

In [ ]:
def noisy_measure(
    counts : dict, 
    probs=gen_error_probs(
        [100,5,0,0,0], # Const - Controls correlation of error weights
        [0, 0, 0, 0, 0], # 1 -> 0 - Controls error biases
        [0, 0, 0, 0, 0] # 0 -> 1 - Controls error biases
        ),
    n_qubits=4
    ) -> dict:
    
    '''
        noisy_measure
        Simulated noisy measurement
        
        :: counts   : dict :: Measurement results
        :: probs    : list :: Measurement error probabilities
        :: n_qubits : int  :: Number of qubits
        
        Returns a dictionary of measurement results
    '''
    
    # Vector of measurment outcomes
    vec = np.zeros((2 ** n_qubits, 1))
    for i in range(2 ** n_qubits):
        try:
            vec[i][0] = counts[str(bin(i)[2:].zfill(n_qubits))]
        except:
            pass
    
    vals = [bin(i)[2:].zfill(n_qubits) for i in range(2 ** n_qubits)]
    
    n_shots = sum(counts.values())
    
    weights = (probs @ (vec / n_shots)).flatten()
    counts_final = weighted_sample_distribution(weights, vals, n_shots)

    return counts_final



def measurement_error(counts, n_qubits=4, probs = gen_error_probs(
        [100,10,40,1,1], # Const - Controls correlation of error weights
        [0, 4,3,3,3], # 1 -> 0 - Controls error biases
        [0,-5,-5,-5,-5] # 0 -> 1 - Controls error biases
        )) -> dict:

    '''
        Just calls noisy measure
        Probably should be deprecated
    '''
    
    counts_final = noisy_measure(counts, n_qubits=n_qubits, probs=probs)
    
    return counts_final



def cal_res_measurement_error(
    cal_results,
    probs : list,
    n_qubits=4
    ) -> dict:
    
    '''
        cal_res_measurement_error
        Calculates the output after applying a simulated measurement error
        
        :: cal_results : results object :: Results before the measurement error
        :: probs       : list :: Measurement error to apply
        :: n_qubits    : int  :: The number of qubits to apply over
        
        Acts in place on the cal_results object
    '''
    
    # Loop over results and construct counts
    for i, res in enumerate(cal_results.results):
        d = {}
        cd = res.data.to_dict()['counts']
        for key in cd:
            d[bin(int(key, 16))[2:].zfill(n_qubits)] = cd[key]
        
        # Apply measurement errors to the counts
        counts = measurement_error(d, n_qubits=n_qubits, probs=probs)

        # Fix the keys back to hex formatting
        data_counts = {}
        for key in counts:
            data_counts[hex(int(key, 2))] = counts[key]
        cal_results.results[i].data.counts = data_counts
        
    return

## Measurement Error Mitigation Strategies ##

### No Correction ###
Simply run and report the baseline error rate

In [ ]:
def no_correction(circuit, 
                  probs=None, 
                  n_shots=1000,
                  n_qubits=4) -> dict:
    '''
        No correction baseline
        Simply performs the circuits and reports the results
    '''
    tmp_circuit = copy.deepcopy(circuit)
    tmp_circuit = design_circuit(n_qubits, '0' * n_qubits, circuit=tmp_circuit)

    job = execute(tmp_circuit, backend, shots=n_shots)

    results = job.result().get_counts()

    if probs is not None:
        noisy_measurement = measurement_error(results, n_qubits=n_qubits, probs=probs)
        results = sample_distribution(noisy_measurement, n_shots)

    # Result strings are in reverse order
    qiskit_results_qubit_order = {}
    for i in results:
        qiskit_results_qubit_order[i[::-1]] = results[i]
    return qiskit_results_qubit_order

### SIM ###
Average over a set of measurement operators to try to mitigate state dependent errors

In [ ]:
# SIM for even numbers of measured qubits 
def sim(circuit, 
        probs=None,
        n_shots=1000,
        n_qubits=4) -> dict:
    '''
        SIM
        Construct four target measurement inv_arrays and average results over them
        This implementation works for even numbers of measured qubits, sim_strs would need to be modified for 
        the odd case
        
        :: circuit  :: Circuit to perform SIM over
        :: probs    :: Simulated error channel to apply
        :: n_shots  :: Number of shots to take
        :: n_qubits :: Number of qubits to measure
        
        Returns the shot statistics following SIM
    '''

    # Construct inv_arrays
    sim_strs = [
        [0] * n_qubits, 
        [1] * n_qubits,
        [0, 1] * (n_qubits // 2), 
        [1, 0] * (n_qubits // 2)
    ]
    
    # Number of shots per inv_array
    shots = n_shots // len(sim_strs)
    
    sim_results = {}
    for inversion_arr in sim_strs:
    
        # New circuit per inv array
        tmp_circuit = copy.deepcopy(circuit)
        tmp_circuit = design_circuit(n_qubits, inversion_arr, circuit=tmp_circuit)

        job = execute(tmp_circuit, backend, shots=shots)

        results = job.result().get_counts()

        # Apply simulated measurement errors
        if probs is not None:
            noisy_measurement = measurement_error(results, n_qubits=n_qubits, probs=probs)
            results = sample_distribution(noisy_measurement, shots)
        
        # Join results for averaging
        for count in results:
            # Invert SIM strings
            count_arr = ''.join(map(str, [i ^ j for i, j in zip(inversion_arr, map(int, list(count[::-1])))]))

            if count_arr in sim_results:
                sim_results[count_arr] += results[count]
            else:
                sim_results[count_arr] = results[count]

    return sim_results

## AIM ##
Select a subset "top-k" of measurement operators and average over those

In [ ]:
# AIM
def aim(circuit, 
        probs=None, # Simulated error channel
        n_shots=1000, # Number of shots
        n_qubits=4, # Number of qubits to measure
        k=4 # Top k strings
       ) -> dict:
    '''
        AIM
        Performs adaptive invert and measure on a target circuit
        :: circuit  :: Circuit to perform AIM over
        :: probs    :: Simulated error channel to apply
        :: n_shots  :: Number of shots to take
        :: n_qubits :: Number of qubits to measure
        :: k        :: Top k inversion strings to use
        
        Returns the shot statistics following AIM
    '''
    
    confirmation_shots = n_shots // k
    
    
    # Build AIM strings
    aim_strs = [[0] * n_qubits for _ in range(n_qubits)]
    
    # Running on four qubits here, so rather than [0000, 1111] we'll run 1000, 0100 etc
    # Could probably compare to 1100, 0110, 0011, 1001
    for i in range(len(aim_strs)):
        aim_strs[i][i] = 1
    aim_strs += [[0] * n_qubits]

    # Number of shots per string
    shots = n_shots // len(aim_strs)
    
    aim_results = {}
    for inversion_arr in aim_strs:
    
        # Build and execute AIM circuits
        tmp_circuit = copy.deepcopy(circuit)
        tmp_circuit = design_circuit(n_qubits, inversion_arr, circuit=tmp_circuit)
        job = execute(tmp_circuit, backend, shots=shots)
        results = job.result().get_counts()

        # Simulated Noisy Measurement
        if probs is not None:
            noisy_measurement = measurement_error(results, n_qubits=n_qubits, probs=probs)
            results = sample_distribution(noisy_measurement, shots)

        # Invert AIM Strings
        aim_result = {}
        for count in results:
            count_arr = ''.join(map(str, [i ^ j for i, j in zip(inversion_arr, map(int, list(count[::-1])))]))

            if count_arr in aim_result:
                aim_result[count_arr] += results[count]
            else:
                aim_result[count_arr] = results[count]

        aim_results[''.join(map(str, inversion_arr))] = aim_result
    
    # Join across inversion strings
    likelihoods = {}
    for res in aim_results:
        for state in aim_results[res]:
            if state in likelihoods:
                likelihoods[state] += aim_results[res][state]
            else:
                likelihoods[state] = aim_results[res][state]
    
    # Select top k strings
    top_k = []
    for i in range(k):
        top = max(likelihoods.items(), key=lambda i: i[1])
        top_k.append(top[0])
        likelihoods.pop(top[0])
    

    # Run confirmation shots for statistics
    tmp_circuits = [copy.deepcopy(circuit) for _ in range(k)]
    tmp_circuits = [design_circuit(n_qubits, i, circuit=t) for i, t in zip(top_k, tmp_circuits)]

    # Run final
    aim_result = {}
    for circ, inversion_arr in zip(tmp_circuits, top_k):

        job = execute(circ, backend, shots=confirmation_shots)
        results = job.result().get_counts()

        # Simulated Noisy Measurement
        if probs is not None:
            noisy_measurement = measurement_error(results, n_qubits=n_qubits, probs=probs)
            results = sample_distribution(noisy_measurement, confirmation_shots)

        for count in results:
            # Invert measurement results using k strings
            count_arr = ''.join(
                map(str, [i ^ j for i, j in zip(map(int, list(inversion_arr)), map(int, list(count[::-1])))])
            )

            if count_arr in aim_result:
                aim_result[count_arr] += results[count]
            else:
                aim_result[count_arr] = results[count]

    return aim_result

### Tensor Calibration ###
Calibrate each qubit individually, then tensor the calibration matricies
Performs well against state dependence, but cannot detect correlations

In [ ]:
def ibmq_sliding_filter(circuit, probs=None, n_shots=1000, n_qubits=4) -> dict:
    '''
        ibmq_sliding_filter
        Performs tensored measurement error calibration on a target circuit
        :: circuit  :: Circuit to perform tensor calibration over
        :: probs    :: Simulated error channel to apply
        :: n_shots  :: Number of shots to take
        :: n_qubits :: Number of qubits to measure
        
        Returns the shot statistics following measurement error calibration
    '''

    # Shots per calibration circuit, 50% of shots for building the filter
    shots = n_shots // (2 ** (n_qubits)) 
    
    # Confirmation shots after calibration
    confirmation_shots = n_shots 
    
    # Build Calibration Matrix
    if n_qubits % 2:
        mit_pattern = [[i, i + 1] for i in range(0, n_qubits - 3, 2)] 
        mit_pattern += [[n_qubits - 3, n_qubits - 2, n_qubits - 1]]
    else:
        mit_pattern = [[i, i + 1] for i in range(0, n_qubits, 2)]
        
    qr = QuantumRegister(n_qubits)
    meas_calibs, state_labels = tensored_meas_cal(mit_pattern=mit_pattern, qr=qr, circlabel='mcal')

    t_qc = transpile(meas_calibs, backend)
    qobj = assemble(t_qc, shots = shots)
    cal_results = backend.run(qobj, shots=n_shots).result()

    # Apply measurement errors to cal results
    if probs is not None:
        cal_res_measurement_error(cal_results, probs, n_qubits=n_qubits)

    # Pass results to the IBM fitter
    meas_fitter = TensoredMeasFitter(cal_results, state_labels, circlabel='mcal')

    # Add measurements to circuit
    circuit = design_circuit(n_qubits, '0' * n_qubits, circuit=circuit)

    # Once the fitter is constructed, attempt the real experiments
    job = execute(circuit, backend, shots=confirmation_shots)
    result = job.result()
    
    # Simulated measurement errors
    if probs is not None:
        cal_res_measurement_error(result, probs, n_qubits=n_qubits)
    
    qiskit_results = meas_fitter.filter.apply(result).get_counts()
    
    # Results are in reverse order
    qiskit_results_qubit_order = {}
    for i in qiskit_results:
        qiskit_results_qubit_order[i[::-1]] = qiskit_results[i]
    return qiskit_results_qubit_order
    

### Full Calibration ###
Basically process tomography for the measurement basis, scales exponentially in the number of qubits

In [ ]:
def ibmq_filter(circuit, n_shots=1000, probs=None, n_qubits=4) -> dict:
    '''
        ibmq_filter
        Performs all qubit measurement error calibration on a target circuit
        
        :: circuit  :: Circuit to perform measurement error calibration over
        :: probs    :: Simulated error channel to apply
        :: n_shots  :: Number of shots to take
        :: n_qubits :: Number of qubits to measure
        
        Returns the shot statistics following calibration
    '''
    
    # Shots per calibration circuit, 50% of shots for building the filter
    shots = n_shots // (2 ** (n_qubits)) 
    
    # Confirmation shots to use with the filter
    confirmation_shots = n_shots 
    
    # Build Calibration Matrix
    qr = QuantumRegister(4)
    meas_calibs, state_labels = complete_meas_cal(qr=qr, circlabel='mcal')

    t_qc = transpile(meas_calibs, backend)
    qobj = assemble(t_qc, shots = shots)
    cal_results = backend.run(qobj, shots=shots).result()
    
    if probs is not None:
        # Apply measurement biases to cal results
        cal_res_measurement_error(cal_results, probs, n_qubits=n_qubits)
    
    # Pass results to the IBM fitter
    meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='mcal')

    # Add measurements to circuit
    circuit = design_circuit(n_qubits, '0' * n_qubits, circuit=circuit)

    # Once the fitter is constructed, attempt the real experiments
    job = execute(circuit, backend, shots=confirmation_shots)
    result = job.result()
    
    if probs is not None:
        cal_res_measurement_error(result, probs, n_qubits=n_qubits)

    result = meas_fitter.filter.apply(result).get_counts()
    
    # Results are in reverse order
    qiskit_results_qubit_order = {}
    for i in result:
        qiskit_results_qubit_order[i[::-1]] = result[i]
    return qiskit_results_qubit_order
    

### Coupling Map Calibration [New Method] ###
Performs calibration over adjacent pairs on the coupling map
Better scaling than exponential while preserving local correlations

In [ ]:
def composite_map(backend, n_qubits=4, probs=None, n_shots=1000, assert_accuracy=False, coupling_map=None):
    '''
        Build a composite map fitter given a backend and a coupling map
        
        :: backend  :: Backend object with associated coupling map
        :: probs    :: Simulated error channel to apply
        :: n_shots  :: Number of shots to take
        :: n_qubits :: Number of qubits to measure
        :: assert_accuracy :: Check accuracy of approximation
        :: coupling_map :: Coupling map if backend does not have one
        
        Returns a composite map calibration filter
    '''
    
    # Converts coupling map tuples to integers
    cmap_to_mit = lambda x, y: x * n_qubits + y

    # Build a register for calibrations
    qr = qiskit.QuantumRegister(n_qubits)

    # Fix our qubit layout
    initial_layout = {}
    for i in range(n_qubits):
        initial_layout[qr[i]] = i

    # Unique Assignment and clear duplicates
    mit_patterns = {}
    
    # Coupling map override
    if coupling_map is None:
        coupling_map = backend.configuration().coupling_map
        
    for pat in coupling_map:
        # Cull couplings with qubits that aren't in our register
        out_of_scope = False
        for i in pat:
            if i >= n_qubits:
                out_of_scope = True
        
        # Cull duplicate couplings
        if not out_of_scope:
            if cmap_to_mit(*pat) not in mit_patterns and cmap_to_mit(*pat[::-1]) not in mit_patterns:
                mit_patterns[cmap_to_mit(*pat)] = pat
    
    
    # Number of shots per trial
    shots = n_shots // (len(mit_patterns) * 4)

    # Approximate two qubit error channel for calibration
    if probs is not None:
        pair_probs = np.array(probs)[:4, :4]

    
    qubit_pair_fitters = {}
    for pat in mit_patterns:
        
        # Construct calibration circuits
        meas_calibs, state_labels = tensored_meas_cal(mit_pattern=[mit_patterns[pat]], qr=qr, circlabel='mcal')
        meas_calibs_t = transpile(meas_calibs, initial_layout=initial_layout, optimization_level=0)
        
        # Execute calibration circuits independently
        # Running simultaneously may result in later experiments being shortchanged on shots
        calibration_results = [execute(i, backend, shots=shots).result() for i in meas_calibs_t]
        
        # Join into one results object
        calibration_results[0].results += [i.results[0] for i in calibration_results[1:]]
        calibration_result = calibration_results[0]
        
        # Apply any simulated measurement errors
        if probs is not None:
            cal_res_measurement_error(calibration_result, pair_probs, n_qubits=2)
        
        fitter = TensoredMeasFitter(calibration_result, state_labels, circlabel='mcal')
        qubit_pair_fitters[pat] = fitter.cal_matrices[0]

        
   # Join calibration matrices
    for i in range(n_qubits):

        # Count number of participating matricies
        num_participants = 0
        for j in mit_patterns:
            if i in mit_patterns[j]:
                num_participants += 1

        # Modify for general construction beyond pairs
        participant_num = 0 # Order of construction
        for pat in mit_patterns:
            if i in mit_patterns[pat]:

                position = mit_patterns[pat].index(i)

                cal_matrix = Qobj(qubit_pair_fitters[pat], dims=f_dims(2))

                # Ptrace to approximate the target qubit
                single_qubit_approx = normalise(np.array(cal_matrix.ptrace(position)))

                # Construct left and right approximations
                mean_approx_l = scipy.linalg.fractional_matrix_power(
                    normalise(np.array(single_qubit_approx)),
                    (num_participants - 1 - participant_num) / num_participants
                )

                mean_approx_r = scipy.linalg.fractional_matrix_power(
                    normalise(np.array(single_qubit_approx)),
                    participant_num / num_participants
                )
            
                # Check accuracy of approximation
                if assert_accuracy:
                    assert(np.linalg.norm(
                         mean_approx_l 
                       @ scipy.linalg.fractional_matrix_power(single_qubit_approx, 1 / num_participants)
                       @ mean_approx_r
                       - single_qubit_approx
                    ) < 1e-4)

                # Expand and set to the correct terms, currently in order [a, b, I, I, I ...]
                expanded_approx_l = [np.eye(2) for _ in range(len(mit_patterns[pat]) - 1)]
                expanded_approx_l = np.kron(mean_approx_l, expanded_approx_l)
                expanded_approx_l = Qobj(expanded_approx_l[0], dims=f_dims(len(mit_patterns[pat])))

                expanded_approx_r = [np.eye(2) for _ in range(len(mit_patterns[pat]) - 1)]
                expanded_approx_r = np.kron(mean_approx_r, expanded_approx_r)
                expanded_approx_r = Qobj(expanded_approx_r[0], dims=f_dims(len(mit_patterns[pat])))

                # Construct permutation order
                order = list(range(len(mit_patterns[pat])))
                order[position] = 0
                order[0] = position

                # Permute to correct order
                expanded_approx_l = expanded_approx_l.permute(order)
                expanded_approx_r = expanded_approx_r.permute(order)

                # Convert back to numpy array
                expanded_approx_l = np.array(expanded_approx_l)
                expanded_approx_r = np.array(expanded_approx_r)

                qubit_pair_fitters[pat] = (
                      np.linalg.inv(expanded_approx_l) 
                    @ qubit_pair_fitters[pat] 
                    @ np.linalg.inv(expanded_approx_r)
                )

                participant_num += 1

    # Doesn't need sparse matricies for small devices
    for pat in mit_patterns:

        pair = mit_patterns[pat]
        pair_approx = qubit_pair_fitters[pat]

        expanded_approx = reduce(np.kron, [pair_approx] + [np.eye(2)] * (n_qubits - len(pair)))
        expanded_approx = Qobj(expanded_approx, dims=[[2 for i in range(n_qubits)]] * 2)

        # Construct ordering for permutation
        # First n elements of the expanded approximation are non-identity and need to be correctly swapped
        # Last k elements are all the identity and may be freely interchanged
        order = []
        order_count = len(pair)
        pair_count = 0
        for i in range(n_qubits):
            if i in pair:
                order.append(pair_count)
                pair_count += 1
            else:
                order.append(order_count)
                order_count += 1

        # Apply permutation
        expanded_approx = np.array(expanded_approx.permute(order))
        qubit_pair_fitters[pat] = expanded_approx

    # Base calibration matrix
    cal_matrix = np.eye(2 ** n_qubits)
    for pat in mit_patterns:
        cal_matrix = qubit_pair_fitters[pat] @ cal_matrix
    
    cal_matrix = np.real(cal_matrix)
        
    # Build new cal matrix object:
    state_labels = [str(bin(i)[2:]).zfill(n_qubits) for i in range(2 ** n_qubits)]
    fitter = CompleteMeasFitter(results=None, state_labels=state_labels)
    
    # Set the corresponding objects appropriately
    fitter._tens_fitt.cal_matrices = [cal_matrix]
    return fitter


def composite_filter(circuit, probs=None, n_shots=1000, n_qubits=4, **kwargs):
    '''
        composite_filter
        Performs all qubit measurement error calibration on a target circuit 
        Using coupling map pairs to build a composite filter
        
        :: circuit  :: Circuit to perform measurement error calibration over
        :: probs    :: Simulated error channel to apply
        :: n_shots  :: Number of shots to take
        :: n_qubits :: Number of qubits to measure
        
        Returns the shot statistics following calibration
    '''
    
    
    shots = n_shots # 50% of shots on building the filter
    confirmation_shots = n_shots 
    
    # Build Calibration Matrix
    qr = QuantumRegister(n_qubits)
    meas_calibs, state_labels = complete_meas_cal(qr=qr, circlabel='mcal')
    
    # Build the filter
    comp_filter = composite_map(backend, n_qubits=n_qubits, probs=probs, n_shots=shots, **kwargs)
    
    # Add measurements to circuit
    circuit = design_circuit(n_qubits, '0' * n_qubits, circuit=circuit)

    # Once the fitter is constructed, attempt the real experiments
    job = execute(circuit, backend, shots=confirmation_shots)
    result = job.result()
    
    if probs is not None:
        cal_res_measurement_error(result, probs, n_qubits=n_qubits)
        
    result = comp_filter.filter.apply(result).get_counts()
    
    # Results are in reverse order
    qiskit_results_qubit_order = {}
    for i in result:
        qiskit_results_qubit_order[i[::-1]] = result[i]
    return qiskit_results_qubit_order



def normalise(x):
    '''
        Normalise the partial trace of a calibration matrix
    '''
    for i in range(x.shape[1]):
        tot = sum(x[:, i])
        if tot != 0:
            x[:, i] /= tot
    return x

def f_dims(n):
    '''
        Dimension ordering for n qubits
    '''
    return [[2 for i in range(n)]] * 2
    

## Experiment Runner ##
Runs an error mitigation method over an optional simulated error channel and a number of shots

In [ ]:
def scale_shots_bv(
    shots_vec : list, # Vector of shots
    fn : Callable,    # Error mitigation function
    *args,
    probs=None,       # Simulated error probabilities
    n_qubits=4,       # Number of qubits
    prog=True,        # Progress Bar
    plabel='Scaling', # Progress Bar Label
    **kwargs
):
    
    '''
        Apply error mitigation methods over a range of total numbers of shots
        
        :: fn        :: Error mitigation method
        :: shots_vec :: Vector of a range of numbers of shots
        :: probs     :: Simulated measurement errors 
        :: n_qubits  :: Number of qubits to measure
        :: prog      :: Progress bar
        :: plabel    :: Label on progress bar
        
        Returns the data along with the number of shots associated with each point
    '''
    
    data = []

    # Progress bar
    prog = Pbar(20, 2 ** n_qubits, plabel)
    
    # For each bv string
    for b_str in range(2 ** n_qubits):
        
        # Construct BV target string
        bv_string = bin(b_str)[2:].zfill(n_qubits)    
        
        # Tick progress bar
        prog(bv_string)
        
        # Construct circuit
        circuit = bv_circuit(bv_string, n_qubits + 1)

        # Apply shots over the range of shots
        scale = map(lambda i : fn(circuit, *args, probs=probs, n_shots=i, **kwargs), shots_vec)
        
        # Collect data
        data += list(np.array([i[bv_string] if bv_string in i else 0 for i in scale]) / np.array(shots_vec))
                
    x_points = np.kron(np.ones(2 ** n_qubits), np.array(shots_vec))
    print()
    return x_points, data

# Experiments #

#### Experiment: ####
Simulated error channel with distance 1 errors

Will take several hours to run.

In [ ]:
backend = Aer.get_backend("qasm_simulator")

n_qubits = 4
probs = gen_error_probs(
        [100,5,0,0,0], # Const - Controls correlation of error weights
        [0, 0, 0, 0, 0], # 1 -> 0 - Controls error biases
        [0, 0, 0, 0, 0] # 0 -> 1 - Controls error biases
        )
shots_vec = range(1000, 16000, 50)

sbs.heatmap(probs)

# Fake coupling map
coupling_map = [[i, i + 1] for i in range(n_qubits)]

x_points, sim_data_d1 = scale_shots_bv(shots_vec, sim, probs=probs, plabel='SIM')
x_points, aim_data_d1 = scale_shots_bv(shots_vec, aim, probs=probs, plabel='AIM')
x_points, ibmq_data_d1 = scale_shots_bv(shots_vec, ibmq_filter, probs=probs, plabel='IBMQ')
x_points, ibmqs_data_d1 = scale_shots_bv(shots_vec, ibmq_sliding_filter, probs=probs, plabel='IBMQ Sliding')
x_points, comp_data_d1 = scale_shots_bv(shots_vec, composite_filter, probs=probs, coupling_map=coupling_map, plabel='CMAP')
x_points, base_data_d1 = scale_shots_bv(shots_vec, no_correction, probs=probs, plabel='Base')


#### Plotting ####
Histplots of the results

In [ ]:
shots_vec = range(1000, 16000, 50)

x_points = np.kron(np.ones(2 ** n_qubits), np.array(shots_vec))

ax = sbs.histplot(x=x_points, y=base_data_d1, color='green')
sbs.histplot(x=x_points, y=np.array(ibmq_data_d1), color='orange')
sbs.histplot(x=x_points, y=np.array(base_data_d1), color='green')
sbs.histplot(x=x_points, y=sim_data_d1)
sbs.histplot(x=x_points, y=np.array(aim_data_d1), color='red')

#ax.set(xlim=(0, 16000))
plt.title('Distance 1 IID Errors')
plt.xlabel("Total Measurements")
plt.ylabel("Success Probability")

In [ ]:
n_qubits = 4
x_points = np.kron(np.ones(2 ** n_qubits), np.array(shots_vec))
ax = sbs.histplot(x=x_points, y=np.array(base_data_d1), color='green')
#ax.set(ylim=(0.799, 1.01))

#plt.text(7000, 0.86, 'Unmitigated')

plt.title('Distance 1 IID Errors')
plt.xlabel("Total Shots")
plt.ylabel("Success Probability")

#plt.savefig('1_IID_base.pdf')

In [ ]:
n_qubits = 4
x_points = np.kron(np.ones(2 ** n_qubits), np.array(shots_vec))
ax = sbs.histplot(x=x_points, y=np.array(aim_data_d1), color='red')
sbs.histplot(x=x_points, y=np.array(ibmq_data_d1), color='orange', ax=ax)
ax.set(ylim=(0.799, 1.01))

#plt.text(6000, 0.97, 'All Qubit Calibration')
#plt.text(8000, 0.86, 'AIM')

plt.title('Distance 1 IID Errors')
plt.xlabel("Total Shots")
plt.ylabel("Success Probability")

#plt.savefig('1_IID_aim_all.pdf')

In [ ]:
n_qubits = 4
x_points = np.kron(np.ones(2 ** n_qubits), np.array(shots_vec))
ax = sbs.histplot(x=x_points, y=np.array(sim_data_d1), color='blue', label='SIM')
sbs.histplot(x=x_points, y=np.array(ibmqs_data_d1), color='yellow', label='Linear Calibration', ax=ax)
ax.set(ylim=(0.799, 1.01))

plt.title('Distance 1 IID Errors')
plt.xlabel("Total Shots")
plt.ylabel("Success Probability")

#plt.text(6000, 0.96, 'Single Qubit Calibration')
#plt.text(8000, 0.86, 'SIM')

#plt.savefig('1_IID_sim_lin.pdf')

In [ ]:
n_qubits = 4
x_points = np.kron(np.ones(2 ** n_qubits), np.array(shots_vec))
ax = sbs.histplot(x=x_points, y=np.array(base_data_d1), color='green')
ax.set(ylim=(0.799, 1.01))

plt.title('Distance 1 IID Errors Uncorrected')
plt.xlabel("Total Measurements")
plt.ylabel("Success Probability")

#### Experiment: ####
Distance 2 error channel

In [ ]:
backend = Aer.get_backend("qasm_simulator")
shots_vec = range(1000, 16000, 50)

probs = gen_error_probs(
        [100, 0, 5,0,0], # Const - Controls correlation of error weights
        [0, 0, 0, 0, 0], # 1 -> 0 - Controls error biases
        [0, 0, 0, 0, 0] # 0 -> 1 - Controls error biases
        )

# Fake coupling map
coupling_map = [[i, i + 1] for i in range(n_qubits)]

# Plot heatmap
sbs.heatmap(probs)

x_points, sim_data_d2 = scale_shots_bv(shots_vec, sim, probs=probs, plabel='SIM')
x_points, aim_data_d2 = scale_shots_bv(shots_vec, aim, probs=probs, plabel='AIM')
x_points, ibmq_data_d2 = scale_shots_bv(shots_vec, ibmq_filter, probs=probs, plabel='IBMQ')
x_points, ibmqs_data_d2 = scale_shots_bv(shots_vec, ibmq_sliding_filter, probs=probs, plabel='IBMQ Sliding')
x_points, comp_data_d2 = scale_shots_bv(shots_vec, composite_filter, probs=probs, coupling_map=coupling_map, plabel='CMAP')
x_points, base_data_d2 = scale_shots_bv(shots_vec, no_correction, probs=probs, plabel='Base')

#### Plotting ####

In [ ]:
ax = sbs.histplot(x=x_points, y=base_data_d2, color='green')
sbs.histplot(x=x_points, y=np.array(sim_data_d2), color='blue', ax=ax)
sbs.histplot(x=x_points, y=np.array(aim_data_d2), color='red', ax=ax)
sbs.histplot(x=x_points, y=np.array(ibmq_data_d2), color='orange', ax=ax)
#ax.set(xlim=(0, 16000), ylim=(0.6, 1.0))
plt.title('Distance 2 Correlated Errors')
plt.xlabel("Total Measurements")
plt.ylabel("Success Probability")

#### Experiment: ####
Single qubit state dependent errors

In [ ]:
backend = Aer.get_backend("qasm_simulator")
shots_vec = range(1000, 16000, 50)

probs = gen_error_probs(
        [100, 1, 0,0,0], # Const - Controls correlation of error weights
        [0, 1, 0, 0, 0], # 1 -> 0 - Controls error biases
        [0, 0, 0, 0, 0] # 0 -> 1 - Controls error biases
        )
sbs.heatmap(probs)

# Fake coupling map for simulator
coupling_map = [[i, i + 1] for i in range(n_qubits)]


x_points, sim_data_b1 = scale_shots_bv(shots_vec, sim, probs=probs, plabel='SIM')
x_points, aim_data_b1 = scale_shots_bv(shots_vec, aim, probs=probs, plabel='AIM')
x_points, ibmq_data_b1 = scale_shots_bv(shots_vec, ibmq_filter, probs=probs, plabel='IBMQ')
x_points, ibmqs_data_b1 = scale_shots_bv(shots_vec, ibmq_sliding_filter, probs=probs, plabel='IBMQ Sliding')
x_points, comp_data_b1 = scale_shots_bv(shots_vec, composite_filter, probs=probs, coupling_map=coupling_map, plabel='CMAP')
x_points, base_data_b1 = scale_shots_bv(shots_vec, no_correction, probs=probs, plabel='Base')

In [ ]:
x_points = np.kron(np.ones(2 ** n_qubits), np.array(shots_vec))
ax=sbs.histplot(x=x_points, y=base_data_b1, color='green')
#sbs.histplot(x=x_points, y=sim_data_b1, color="blue", ax=ax)
#sbs.histplot(x=x_points, y=aim_data_b1, color="red", ax=ax)
#sbs.histplot(x=x_points, y=ibmq_data_b1, color='orange', ax=ax)

#plt.text(7000, 0.97, 'Unmitigated')
ax.set(ylim=(0.83, 1.0))

plt.title('Simulated State Dependent Measurement Errors')
plt.xlabel("Total Shots")
plt.ylabel("Success Probability")
#plt.savefig('biased_based.pdf')

In [ ]:
x_points = np.kron(np.ones(2 ** n_qubits), np.array(shots_vec))
#ax=sbs.histplot(x=x_points, y=sim_data_b1, color='blue')
ax=sbs.histplot(x=x_points, y=sim_data_b1, color="blue")
sbs.histplot(x=x_points, y=aim_data_b1, color="red", ax=ax)
sbs.histplot(x=x_points, y=ibmq_data_b1, color='orange', ax=ax)

# plt.text(7000, 0.98, 'Calibration')
# plt.text(8000, 0.948, 'SIM')
# plt.text(8000, 0.87, 'AIM')
ax.set(ylim=(0.83, 1.0))

plt.title('State Dependent Measurement Error Mitigation')
plt.xlabel("Total Shots")
plt.ylabel("Success Probability")
plt.savefig('biased_suppression.pdf')

In [ ]:
print("Mean Error Rates Single Qubit State Dependent Noise")
print('Base: \t\t\t\t', 1 - np.mean(base_data_b1))
print('SIM: \t\t\t\t', 1 - np.mean(sim_data_b1))
print('AIM: \t\t\t\t', 1 - np.mean(aim_data_b1))
print('Calibration: \t\t\t',  1 - np.mean(ibmq_data_b1))
print('Tensor Calibration: \t\t',  1 - np.mean(ibmqs_data_b1))
print('Coupling Map Calibration: \t',  1 - np.mean(comp_data_b1))

#### Experiment ####
Single qubit errors with small state dependence

In [ ]:
shots_vec = range(100, 16000, 100)

probs = gen_error_probs(
        [100, 1, 0,0,0], # Const - Controls correlation of error weights
        [0, 1, 0, 0, 0], # 1 -> 0 - Controls error biases
        [0, 0, 0, 0, 0] # 0 -> 1 - Controls error biases
        )
sbs.heatmap(probs)
x_points, sim_data_b2 = scale_shots_bv(shots_vec, sim, probs=probs, plabel='SIM')
x_points, aim_data_b2 = scale_shots_bv(shots_vec, aim, probs=probs, plabel='AIM')
x_points, ibmq_data_b2 = scale_shots_bv(shots_vec, ibmq_filter, probs=probs, plabel='IBMQ')
x_points, ibmqs_data_b2 = scale_shots_bv(shots_vec, ibmq_sliding_filter, probs=probs, plabel='IBMQ Sliding')
x_points, comp_data_b2 = scale_shots_bv(shots_vec, composite_filter, probs=probs, coupling_map=coupling_map, plabel='CMAP')
x_points, base_data_b2 = scale_shots_bv(shots_vec, no_correction, probs=probs, plabel='Base')

In [ ]:
print("Mean Error Rates Single Qubit State Dependent + Single Qubit IID Noise")
print('Base: \t\t\t\t', 1 - np.mean(base_data_b2))
print('SIM: \t\t\t\t', 1 - np.mean(sim_data_b2))
print('AIM: \t\t\t\t', 1 - np.mean(aim_data_b2))
print('Calibration: \t\t\t',  1 - np.mean(ibmq_data_b2))
print('Tensor Calibration: \t\t',  1 - np.mean(ibmqs_data_b2))
print('Coupling Map Calibration: \t',  1 - np.mean(comp_data_b2))

#### Experiment ####
Just testing a very messy correlated and state dependent set of errors

In [ ]:
shots_vec = range(100, 16000, 100)

probs = gen_error_probs(
        [100,0,10,0,0], # Const - Controls correlation of error weights
        [0, 5, 30, 5, 5], # 1 -> 0 - Controls error biases
        [0,-20,-20,-20,-20] # 0 -> 1 - Controls error biases
        )

sbs.heatmap(probs)

x_points, sim_data_g1 = scale_shots_bv(shots_vec, sim, probs=probs, plabel='SIM')
x_points, aim_data_g1 = scale_shots_bv(shots_vec, aim, probs=probs, plabel='AIM')
x_points, ibmq_data_g1 = scale_shots_bv(shots_vec, ibmq_filter, probs=probs, plabel='IBMQ')
x_points, ibmqs_data_g1 = scale_shots_bv(shots_vec, ibmq_sliding_filter, probs=probs, plabel='IBMQ Sliding')
x_points, comp_data_g1 = scale_shots_bv(shots_vec, composite_filter, probs=probs, coupling_map=coupling_map, plabel='CMAP')
x_points, base_data_g1 = scale_shots_bv(shots_vec, no_correction, probs=probs, plabel='Base')

In [ ]:
sbs.set(style="darkgrid")
shots_vec = range(100, 16000, 100)

x_points = np.kron(np.ones(2 ** n_qubits), np.array(shots_vec))
ax=sbs.histplot(x=x_points, y=base_data_g1, color='green')
sbs.histplot(x=x_points, y=ibmq_data_g1, color='orange', ax=ax)
#sbs.histplot(x=x_points, y=sim_data_g1, color="blue", ax=ax)


#plt.text(6500, 0.9, 'Calibration Full')
#plt.text(8000, 0.8, 'SIM')
#plt.text(6500, 0.75, 'Unmitigated')

#plt.text()
#plt.text(8000, 0.87, 'AIM')

ax.set(ylim=(0.3, 1.0))

plt.title('Correlated and State Dependent Errors')
plt.xlabel("Total Measurements")
plt.ylabel("Success Probability")
plt.savefig("example_2_base.pdf")

In [ ]:
sbs.set(style="darkgrid")
shots_vec = range(100, 16000, 100)

x_points = np.kron(np.ones(2 ** n_qubits), np.array(shots_vec))
ax=sbs.histplot(x=x_points, y=comp_data_g1, color='yellow')
sbs.histplot(x=x_points, y=aim_data_g1, color="red", ax=ax)
sbs.histplot(x=x_points, y=sim_data_g1, color="blue", ax=ax)

#plt.text(4500, 0.93, 'Coupling Map Calibration')
#plt.text(7500, 0.85, 'AIM')
#plt.text(7500, 0.60, 'SIM')

#plt.grid(b=None)

ax.set(ylim=(0.3, 1.0))
plt.title('Correlated and State Dependent Error Mitigation')
plt.xlabel("Total Measurements")
plt.ylabel("Success Probability")
#plt.savefig("example_2_comparisons.pdf")

In [ ]:
print("Mean Error Rates Single Qubit State Dependent + Single Qubit IID Noise")
print('Base: \t\t\t\t', 1 - np.mean(base_data_g1))
print('SIM: \t\t\t\t', 1 - np.mean(sim_data_g1))
print('AIM: \t\t\t\t', 1 - np.mean(aim_data_g1))
print('Calibration: \t\t\t',  1 - np.mean(ibmq_data_g1))
print('Tensor Calibration: \t\t',  1 - np.mean(ibmqs_data_g1))
print('Coupling Map Calibration: \t',  1 - np.mean(comp_data_g1))